In [16]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmcv.runner import load_checkpoint
from tqdm.notebook import tqdm
import torch
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.6.0].
device:[cuda:1].


In [17]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_small.py')

PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=42
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/swin+cascade+small'


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [18]:
import os
from mmcv.runner import load_checkpoint
model = build_detector(cfg.model)
checkpoint_path = os.path.join('./work_dirs/swin+cascade','cascade_mask_rcnn_swin_small_patch4_window7.pth')
checkpoint = load_checkpoint(model, checkpoint_path)

2021-05-15 12:56:29,283 - mmdet - INFO - load model from: /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade/swin_small_patch4_window7_224.pth
2021-05-15 12:56:30,056 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param wi

In [19]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=4.54s)
creating index...
index created!


In [20]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-15 12:56:55,214 - mmdet - INFO - Start running, host: ubuntu@nipa2021-1083, work_dir: /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade+small
2021-05-15 12:56:55,215 - mmdet - INFO - workflow: [('train', 1)], max: 15 epochs


Done (t=0.97s)
creating index...
index created!


2021-05-15 12:57:38,623 - mmdet - INFO - Epoch [1][50/655]	lr: 9.890e-06, eta: 2:21:25, time: 0.868, data_time: 0.077, memory: 16080, loss_rpn_cls: 0.5125, loss_rpn_bbox: 0.0810, s0.loss_cls: 2.2043, s0.acc: 35.8525, s0.loss_bbox: 0.3296, s0.loss_mask: 1.2078, s1.loss_cls: 1.0718, s1.acc: 51.8174, s1.loss_bbox: 0.0427, s1.loss_mask: 0.4885, s2.loss_cls: 0.5482, s2.acc: 30.7803, s2.loss_bbox: 0.0044, s2.loss_mask: 0.3538, loss: 6.8447, grad_norm: 32.5844
2021-05-15 12:58:19,412 - mmdet - INFO - Epoch [1][100/655]	lr: 1.988e-05, eta: 2:16:27, time: 0.816, data_time: 0.026, memory: 16080, loss_rpn_cls: 0.2211, loss_rpn_bbox: 0.0594, s0.loss_cls: 1.0373, s0.acc: 88.3662, s0.loss_bbox: 0.4071, s0.loss_mask: 0.8454, s1.loss_cls: 0.4432, s1.acc: 94.9990, s1.loss_bbox: 0.0560, s1.loss_mask: 0.3816, s2.loss_cls: 0.2440, s2.acc: 97.1270, s2.loss_bbox: 0.0059, s2.loss_mask: 0.2335, loss: 3.9345, grad_norm: 17.0909
2021-05-15 12:59:00,927 - mmdet - INFO - Epoch [1][150/655]	lr: 2.987e-05, eta: 2:1

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 7.0 task/s, elapsed: 94s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.77s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.96s).
Accumulating evaluation results...


2021-05-15 15:56:29,455 - mmdet - INFO - Evaluating segm...


DONE (t=0.60s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.316
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.427
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.334
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.074
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.297
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.497
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.440
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.440
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.440
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.163
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.435
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.625
Loading and preparing resul

2021-05-15 15:56:35,056 - mmdet - INFO - Epoch(val) [15][655]	bbox_mAP: 0.3160, bbox_mAP_50: 0.4270, bbox_mAP_75: 0.3340, bbox_mAP_s: 0.0740, bbox_mAP_m: 0.2970, bbox_mAP_l: 0.4970, bbox_mAP_copypaste: 0.316 0.427 0.334 0.074 0.297 0.497, segm_mAP: 0.2910, segm_mAP_50: 0.4170, segm_mAP_75: 0.3030, segm_mAP_s: 0.0550, segm_mAP_m: 0.2670, segm_mAP_l: 0.4670, segm_mAP_copypaste: 0.291 0.417 0.303 0.055 0.267 0.467


DONE (t=0.54s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.291
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.417
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.303
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.055
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.267
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.467
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.407
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.407
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.407
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.126
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.398
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.597
